In [1]:
from transformers import GPT2Model, GPT2Tokenizer
import torch

In [ ]:
gpt = GPT2Model.from_pretrained("gpt2").cuda()

In [ ]:
gpt.config

In [ ]:
gpt

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.tokenize("It is test.")

In [7]:
tokenizer.encode("It is test.")

[1026, 318, 1332, 13]

In [8]:
tokenizer.vocab_size

50257

In [9]:
inputs = torch.randint(0, 50257, size=(4, 100), dtype=torch.int64).cuda()
pad_mask = (inputs != -100)
outputs = gpt(inputs, attention_mask=pad_mask)

In [10]:
outputs[0].shape

torch.Size([4, 100, 768])

In [11]:
import torch
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer

In [12]:
tok_path = get_tokenizer()

using cached model


In [13]:
tok_path

'/home/jhm9507/kogpt2/kogpt2_news_wiki_ko_cased_818bfa919d.spiece'

In [41]:
tok = SentencepieceTokenizer(tok_path, alpha=0)

In [14]:
from sentencepiece import SentencePieceProcessor as sp

In [15]:
tokenizer = sp(tok_path)

In [16]:
tokenizer.EncodeAsPieces("이것은 테스트 문장입니다. This is text example...")

['▁이것은',
 '▁테스트',
 '▁문장',
 '입니다',
 '.',
 '▁Th',
 'is',
 '▁is',
 '▁',
 'text',
 '▁e',
 'x',
 'amp',
 'le',
 '...']

In [17]:
model, vocab = get_pytorch_kogpt2_model()

using cached model
using cached model


In [18]:
vocab[tokenizer.EncodeAsPieces("이것은 테스트 문장입니다. This is text example...")]

[5983,
 5054,
 24680,
 484,
 47440,
 22047,
 520,
 6213,
 47437,
 9668,
 3276,
 48038,
 19088,
 622,
 218]

In [31]:
tokenizer.EncodeAsIds("이것은 테스트 문장입니다. This is text example...")

[5983,
 5054,
 24680,
 484,
 47440,
 22047,
 520,
 6213,
 47437,
 9668,
 3276,
 48038,
 19088,
 622,
 218]

In [20]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [21]:
inputs = torch.tensor([0] + tokenizer.EncodeAsIds("안녕하세요. 저는")).unsqueeze(dim=0)

In [22]:
pred = model(inputs)[0]

In [23]:
pred.shape

torch.Size([1, 4, 50000])

In [24]:
word = pred[:, -1, :].argmax(dim=-1)

In [25]:
tokenizer.DecodeIds(word.tolist())

'지금'

In [82]:
words = "2019년 한해를 보내며,"
while True:
    inputs = torch.tensor([vocab[vocab.bos_token]] + vocab[tok(words)]).unsqueeze(dim=0)
    pred = model(inputs)[0]
    word = pred[:, -1, :].argmax(dim=-1)
    word = vocab.to_tokens(word.tolist())[-1]
    if word == "</s>":
        break
    print(word)
    words += word.replace("▁", " ")
words

▁새해
에는
▁더
▁많은
▁사람들이
▁새해
에
▁이루고
자
▁하는
▁소망
과
▁희망을
▁되새
겨
보는
▁시간이
▁되었
으면
▁좋겠다
.


'2019년 한해를 보내며, 새해에는 더 많은 사람들이 새해에 이루고자 하는 소망과 희망을 되새겨보는 시간이 되었으면 좋겠다.'

In [83]:
words = "2019년 한해를 보내며,"
words = tokenizer.EncodeAsIds(words)
while True:
    inputs = torch.tensor([0] + words).unsqueeze(dim=0)
    pred = model(inputs)[0]
    word = pred[:, -1, :].argmax(dim=-1)
    print(tokenizer.DecodeIds(word.tolist()))
    word = word[0]
    if word == 1:
        break
    words.append(word.item())
words = tokenizer.DecodeIds(words)
words

새해
에는
더
많은
사람들이
새해
에
이루고
자
하는
소망
과
희망을
되새
겨
보는
시간이
되었
으면
좋겠다
.



'2019년 한해를 보내며, 새해에는 더 많은 사람들이 새해에 이루고자 하는 소망과 희망을 되새겨보는 시간이 되었으면 좋겠다.'